In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import re
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from prophet import Prophet

In [8]:

df = pd.read_csv("datasets/covid_19_data.csv")
df


,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
26708,26709,05/19/2020,Wyoming,US,2020-05-20 02:32:19,776.0,10.0,0.0
26709,26710,05/19/2020,Xinjiang,Mainland China,2020-05-20 02:32:19,76.0,3.0,73.0
26710,26711,05/19/2020,Yukon,Canada,2020-05-20 02:32:19,11.0,0.0,11.0
26711,26712,05/19/2020,Yunnan,Mainland China,2020-05-20 02:32:19,185.0,2.0,183.0


In [9]:
df.dtypes

SNo                  int64
ObservationDate     object
Province/State      object
Country/Region      object
Last Update         object
Confirmed          float64
Deaths             float64
Recovered          float64
dtype: object

In [10]:
#Nomes de colunas não devem ter letras maiúsculas e nem caracteres especiais. Vamos utilizar uma função para corrigir os nomes das colunas.
def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [11]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [12]:
df

,sno,observationdate,provincestate,countryregion,lastupdate,confirmed,deaths,recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
26708,26709,05/19/2020,Wyoming,US,2020-05-20 02:32:19,776.0,10.0,0.0
26709,26710,05/19/2020,Xinjiang,Mainland China,2020-05-20 02:32:19,76.0,3.0,73.0
26710,26711,05/19/2020,Yukon,Canada,2020-05-20 02:32:19,11.0,0.0,11.0
26711,26712,05/19/2020,Yunnan,Mainland China,2020-05-20 02:32:19,185.0,2.0,183.0


In [13]:
#Análises